# Выкачивание данных из базы данных github'a

### Подключение к удаленной базе данных через localhost

In [35]:
import mysql.connector
import numpy as np
import json
import math
from tqdm import tqdm
from datetime import datetime
cnx = mysql.connector.connect(user='ght',
                              host='localhost',
                              database='ghtorrent')
cursor = cnx.cursor()

In [31]:
HH_skill = ['ABAP', 'ASP', 'Assembly', 'C#', 'C++', 'CMake', 'CSS', 'Clojure', 'CoffeeScript', 'ColdFusion', 'Delphi', 'FreeMarker', 'GLSL', 'Go', 'Groovy', 'HTML', 'Handlebars', 'Java', 'JavaScript', 'Kotlin', 'Lua', 'Nginx', 'Objective-C', 'PHP', 'Perl', 'PowerShell', 'Puppet', 'Python', 'Ruby', 'SQL', 'Smarty', 'Swift', 'TypeScript', 'Visual Basic', 'XML', 'XSLT']

### Потоковая запись данных, взятых из базы данных в файл, в формате json

In [22]:
import json
import codecs
with codecs.open('new_data.json', 'a',encoding = 'utf8') as f:
    f.write(']')
def sv(obj,p,flag,encoding="utf8"):
    with codecs.open(p,"a",encoding=encoding) as f:
        if flag == 0:
            f.write('[')
        f.write(json.dumps(obj,indent=1,ensure_ascii=0))
        if flag == 1:
            f.write(']')
            return
        f.write(',')
        
        #json.dumps(,f,indent=1,ensure_ascii=0)

In [17]:
cursor.execute("SELECT * FROM projects where owner_id='%s';" %20)
user_projects = cursor.fetchall()
user_projects[0][1]

'https://api.github.com/repos/metasync/rbczmq'

In [1]:
def get_user_experience(user_id, language):
    """
    user_id - id пользователя в гитхабе.
    language - язык, для которого надо определить опытность данного юзера.
    Для каждого пользователя и языка программирования, на котором он программировал хоть однажды
    Высчитывается его "опыт" программирования на этом языке.
    
    Возвращает число - опыт программиста в программировании на данном языке(language)
    """
    cursor.execute("select * from projects where owner_id='%s' and language = '%s';" %(user_id,language))
    userLang = cursor.fetchall()
    userLang = sorted([i[6] for i in userLang])    
    if len(userLang) == 0:
        return 0
    
    K_N = (userLang[-1].day+userLang[-1].month*30+userLang[-1].year*365) - (userLang[0].day+userLang[0].month*30+userLang[0].year*365) + 1
    
    man = [(q.day,q.year) for q in userLang]
    commitDays = len(list(set(map(tuple, man))))

    return round(math.log(K_N*commitDays)/20+0.31,4)


### Обращение к бд

In [ ]:
main_data = {}
flag = 0
unique_skills = np.array([])
for user in tqdm(range(99000, 100000)):
    
    cursor.execute("SELECT * FROM projects where owner_id='%s';" %user)
    user_projects = cursor.fetchall()

    if(len(user_projects) == 0 or len(user_projects[0]) <= 5):
        continue
    ####[0] - {language: user experience in language}
    ####[1] - count user followers
    ####[2] - count user wathers
    ####[3] - github link from
    main_data[user] = [{}, 0, 0, ''] # вот здесь
   
    user_skills = np.unique([i[5] for i in user_projects if i[5] != None])

    user_link = "https://github.com/" + user_projects[0][1].split('/')[4]
    main_data[user][3] = user_link 
    
    for skill in user_skills:
        if skill in HH_skill:
            main_data[user][0][skill] = get_user_experience(user, skill)
            
    ###
    cursor.execute("SELECT user_id FROM followers where user_id='%s';" %user)
    main_data[user][1] = len(cursor.fetchall())
#     print(main_data[user][1])
    cursor.execute("SELECT user_id FROM watchers where user_id='%s';" %user)
    main_data[user][2] = len(cursor.fetchall())
    if user == 99999:
        flag = 1
    sv(main_data[user],'new_data.json',flag)
    flag = -1
    unique_skills = np.unique(np.append(unique_skills, user_skills))
    main_data = {}

 64%|██████▍   | 645/1000 [15:09<23:52,  4.04s/it]